In [1]:
%pip install exifread folium

  Using cached exifread-3.5.1-py3-none-any.whl.metadata (10 kB)
  Using cached folium-0.20.0-py2.py3-none-any.whl.metadata (4.2 kB)
Using cached exifread-3.5.1-py3-none-any.whl (59 kB)
Using cached folium-0.20.0-py2.py3-none-any.whl (113 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install exifread folium

import exifread
import folium

import base64
import mimetypes
from pathlib import Path

In [3]:

def _ratio_to_float(r):
    return float(r.num) / float(r.den)

def _dms_to_deg(values, ref):
    d = _ratio_to_float(values[0])
    m = _ratio_to_float(values[1])
    s = _ratio_to_float(values[2])
    deg = d + (m / 60.0) + (s / 3600.0)
    if ref in ("S", "W"):
        deg = -deg
    return deg

def extract_gps_from_photo(photo_path):
    with open(photo_path, "rb") as f:
        tags = exifread.process_file(f, details=False)

    lat_vals = tags.get("GPS GPSLatitude")
    lat_ref  = tags.get("GPS GPSLatitudeRef")
    lon_vals = tags.get("GPS GPSLongitude")
    lon_ref  = tags.get("GPS GPSLongitudeRef")

    if not (lat_vals and lat_ref and lon_vals and lon_ref):
        raise ValueError("No EXIF GPS found in this photo.")

    lat = _dms_to_deg(lat_vals.values, str(lat_ref.values))
    lon = _dms_to_deg(lon_vals.values, str(lon_ref.values))
    return lat, lon



In [4]:
# ...existing code...
def make_map_from_media(media_inputs: list, out_html="photo_map_multiple.html", zoom_start=17):
    """
    Accepts a list of inputs (strings or dicts). Each input can be:
      - a string path to an image file (EXIF GPS will be used)
      - a dict with keys:
          - image: path to image file (optional)
          - audio: path to wav/mp3 file (optional)
          - description: short text (optional)
          - lat, lon: numeric coords (optional - required for audio-only items)
    For each item with valid lat/lon the map will add a marker. Clicking the marker
    opens a small popup box that always shows filename + coords and, when available,
    a thumbnail image, description text and an <audio> player (embedded as data URI).
    """
    from html import escape

    items = []

    def _add_item(p_image, p_audio, desc, lat, lon):
        items.append({"image": p_image, "audio": p_audio, "desc": desc, "lat": lat, "lon": lon})

    for entry in media_inputs:
        # normalize to dict
        if isinstance(entry, (str, Path)):
            p = Path(entry)
            ext = p.suffix.lower()
            if ext in {".jpg", ".jpeg", ".png", ".webp", ".gif", ".tiff"}:
                try:
                    lat, lon = extract_gps_from_photo(str(p))
                    _add_item(p, None, None, lat, lon)
                except Exception as e:
                    print(f"Skipping image without GPS: {p.name}: {e}")
            else:
                print(f"Skipping unsupported file (use dict with lat/lon for audio): {p}")
            continue

        if not isinstance(entry, dict):
            print("Unsupported media input, must be path or dict:", entry)
            continue

        p_image = Path(entry["image"]) if entry.get("image") else None
        p_audio = Path(entry["audio"]) if entry.get("audio") else None
        desc = entry.get("description") or entry.get("desc") or ""
        lat = entry.get("lat")
        lon = entry.get("lon")

        # attempt to extract coords from image if not provided
        if (lat is None or lon is None) and p_image:
            try:
                lat, lon = extract_gps_from_photo(str(p_image))
            except Exception:
                lat = lon = None

        # for audio-only entries require lat/lon
        if p_audio and (lat is None or lon is None):
            print(f"Audio item requires explicit lat/lon, skipping: {p_audio.name}")
            continue

        if lat is None or lon is None:
            print(f"No GPS for item, skipping: image={p_image}, audio={p_audio}")
            continue

        _add_item(p_image, p_audio, desc, float(lat), float(lon))

    if not items:
        raise ValueError("No media items with valid coordinates found.")

    # center map
    avg_lat = sum(it["lat"] for it in items) / len(items)
    avg_lon = sum(it["lon"] for it in items) / len(items)
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=zoom_start, tiles="OpenStreetMap")

    for it in items:
        p_image = it["image"]
        p_audio = it["audio"]
        desc = it["desc"] or ""
        lat = it["lat"]
        lon = it["lon"]

        parts = []
        # image thumbnail
        if p_image and p_image.exists():
            try:
                mime, _ = mimetypes.guess_type(str(p_image))
                mime = mime or "image/png"
                b64 = base64.b64encode(p_image.read_bytes()).decode("ascii")
                parts.append(
                    f'<div style="text-align:center;">'
                    f'<img src="data:{mime};base64,{b64}" '
                    f'style="max-width:280px; max-height:180px; width:auto; height:auto; display:block; margin:0 auto; border-radius:4px;" />'
                    f'</div>'
                )
            except Exception as e:
                parts.append(f'<div><em>Failed to embed image: {escape(str(p_image))}</em></div>')

        # description
        if desc:
            parts.append(f'<div style="font-size:12px; margin-top:6px; color:#222;">{escape(desc)}</div>')

        # audio player
        if p_audio and p_audio.exists():
            try:
                amime, _ = mimetypes.guess_type(str(p_audio))
                amime = amime or "audio/wav"
                a64 = base64.b64encode(p_audio.read_bytes()).decode("ascii")
                parts.append(
                    f'<div style="margin-top:6px; text-align:center;">'
                    f'<audio controls style="width:90%;">'
                    f'<source src="data:{amime};base64,{a64}" type="{amime}"/>'
                    "Your browser does not support the audio element."
                    f'</audio></div>'
                )
            except Exception as e:
                parts.append(f'<div><em>Failed to embed audio: {escape(str(p_audio))}</em></div>')

        # filename + coords (always present)
        filename_line = ""
        if p_image:
            filename_line = escape(p_image.name)
        elif p_audio:
            filename_line = escape(p_audio.name)
        parts.append(f'<div style="font-size:11px; margin-top:6px; color:#444; text-align:center;">{filename_line}<br/><span style="font-family:monospace;">{lat:.6f}, {lon:.6f}</span></div>')

        html = "<div>" + "".join(parts) + "</div>"
        # choose iframe size depending on presence of audio
        iframe_height = 260 if p_audio else 200
        iframe = folium.IFrame(html, width=340, height=iframe_height)
        popup = folium.Popup(iframe, max_width=360)
        folium.Marker([lat, lon], popup=popup, tooltip=filename_line or "media").add_to(m)

    # show lat/lon when user clicks on the map
    m.add_child(folium.LatLngPopup())

    try:
        from folium.plugins import MousePosition
        m.add_child(MousePosition(position="topright", separator=" , "))
    except Exception:
        pass

    m.save(out_html)
    return out_html
# ...existing code...

In [7]:
media=[
        {"image":"./IMG_1962.jpg","description":"Nice mural","audio":"./test_01.mp3"},
        {"image":"./IMG_1964.jpg","description":"Nice mural","audio":"./test_01.mp3"}
    ]

make_map_from_media(media)

'photo_map_multiple.html'

In [8]:
# Add a GLB model to the folium map using <model-viewer> inside a popup
# The .glb file should be placed in the same folder as the saved HTML (or be reachable by the browser).
import folium
from html import escape

def add_glb_marker(m, lat, lon, glb_path, title='3D model', width=360, height=360):
    """
    Add a marker to folium Map `m` at (lat, lon) whose popup contains an embedded
    <model-viewer> element that loads `glb_path`. The path is used as-is in the
    generated HTML, so when opening the saved map ensure the GLB is reachable (e.g.
    in the same folder as the saved HTML file).
    """
    # build HTML that loads the model-viewer web component and points to glb_path
    safe_src = escape(str(glb_path))
    safe_title = escape(str(title))
    html = f'''
<script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js"></script>
<style>body{{margin:0;padding:0;background:transparent}} model-viewer{{background:transparent}}</style>
<model-viewer src="{safe_src}" alt="{safe_title}" auto-rotate camera-controls style="width:100%; height:100%;"></model-viewer>
'''
    iframe = folium.IFrame(html, width=width, height=height)
    popup = folium.Popup(iframe, max_width=width+20)
    folium.Marker([lat, lon], popup=popup, tooltip=title).add_to(m)
    return m

In [9]:
# Demo: create a map, add a GLB marker and save to HTML
# Place your model file (e.g. model.glb) next to the saved HTML or give an accessible path.
# Example: put 'model.glb' in the same folder and run this cell; it will write 'map_with_glb.html'.

# center coordinates (change to your coordinates)
demo_lat, demo_lon = 31.2304, 121.4737  # Shanghai approx
m = folium.Map(location=[demo_lat, demo_lon], zoom_start=17, tiles='OpenStreetMap')
# update the path below to point to your GLB file relative to the saved HTML (or absolute URL)
glb_path = './Chifeng_Rd.glb'
add_glb_marker(m, demo_lat, demo_lon, glb_path, title='My GLB Model', width=420, height=360)
out_html = 'map_with_glb.html'
m.save(out_html)
out_html

'map_with_glb.html'

'map_with_glb_http.html'

In [ ]:
# Create an always-visible overlay containing the model-viewer
# This reads an existing saved map HTML (map_with_glb.html or map_with_glb_http.html),
# injects a floating <model-viewer> panel and saves map_with_glb_overlay.html.
from pathlib import Path

candidates = ['map_with_glb_http.html', 'map_with_glb.html']
src_html = None
for c in candidates:
    if Path(c).exists():
        src_html = c
        break
if not src_html:
    raise FileNotFoundError('No base map HTML found. Run the demo cells to create map_with_glb.html first.')

glb_local = './Chifeng_Rd.glb'
# prefer HTTP if present (we started a server earlier)
glb_http = 'http://localhost:8000/Chifeng_Rd.glb'
glb_to_use = glb_http if True else glb_local

overlay = f'''
<!-- START: model-viewer overlay -->
<link rel="stylesheet" href="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.css"/>
<div id="model-overlay" style="position: absolute; right: 18px; bottom: 18px; width: 440px; height: 360px; z-index: 1200; background: rgba(255,255,255,0.95); border-radius:8px; overflow:hidden; box-shadow:0 6px 18px rgba(0,0,0,0.25);">
  <script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js"></script>
  <model-viewer id="mv-overlay" src="{glb_to_use}" alt="Overlay model" camera-controls auto-rotate exposure="1" shadow-intensity="1" style="width:100%; height:100%; background:#ffffff;" reveal="interaction"></model-viewer>
</div>
<script>
document.addEventListener('DOMContentLoaded', function(){
  const mv = document.getElementById('mv-overlay');
  if(!mv) return;
  mv.addEventListener('load', ()=>{
    try{
      // Set a nicer camera orbit so the model is in view.
      mv.cameraOrbit = '0deg 75deg 2.5m';
      mv.jumpCameraToGoal();
    }catch(e){ console.log('model-viewer camera adjust failed', e); }
  });
});
</script>
<!-- END: model-viewer overlay -->
'''

src_text = Path(src_html).read_text(encoding='utf-8')
if '</body>' in src_text:
    new_text = src_text.replace('</body>', overlay + '\n</body>')
else:
    new_text = src_text + overlay
out_path = 'map_with_glb_overlay.html'
Path(out_path).write_text(new_text, encoding='utf-8')
out_path